From previous experiments, we have following thresholds for Feature Squeezing methods: 

For 1% FPR:

- CIFAR: 1.15526
- MNIST: 0.54005
- SVHN: 0.17120

# Feature squeezing functions

In [110]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [111]:
l1_dist = lambda x1,x2: np.sum(np.abs(x1 - x2), axis=tuple(range(len(x1.shape))[1:]))
l2_dist = lambda x1,x2: np.sum((x1-x2)**2, axis=tuple(range(len(x1.shape))[1:]))**.5

In [112]:
# Note: KL-divergence is not symentric.
# Designed for probability distribution (e.g. softmax output).
def kl(x1, x2):
    assert x1.shape == x2.shape
    # x1_2d, x2_2d = reshape_2d(x1), reshape_2d(x2)

    # Transpose to [?, #num_examples]
    x1_2d_t = x1.transpose()
    x2_2d_t = x2.transpose()

    # pdb.set_trace()
    e = entropy(x1_2d_t, x2_2d_t)
    e[np.where(e==np.inf)] = 2
    return e

In [113]:
def reduce_precision_py(x, npp):
    """
    Reduce the precision of image, the numpy version.
    :param x: a float tensor, which has been scaled to [0, 1].
    :param npp: number of possible values per pixel. E.g. it's 256 for 8-bit gray-scale image, and 2 for binarized image.
    :return: a tensor representing image(s) with lower precision.
    """
    # Note: 0 is a possible value too.
    npp_int = npp - 1
    x_int = np.rint(x * npp_int)
    x_float = x_int / npp_int
    return x_float

In [114]:
def median_filter_py(x, width, height=-1):
    """
    Median smoothing by Scipy.
    :param x: a tensor of image(s)
    :param width: the width of the sliding window (number of pixels)
    :param height: the height of the window. The same as width by default.
    :return: a modified tensor with the same shape as x.
    """
    if height == -1:
        height = width
    var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')
    #print("inside median filter")
    #print(type(var))
    return torch.from_numpy(var)

In [115]:
# Squeezers implemented in OpenCV
# OpenCV expects uint8 as image data type.
def opencv_wrapper(imgs, opencv_func, argv):
    ret_imgs = []
    imgs_copy = imgs

    if imgs.shape[3] == 1:
        imgs_copy = np.squeeze(imgs)

    for img in imgs_copy:
        img_uint8 = np.clip(np.rint(img * 255), 0, 255).astype(np.uint8)
        ret_img = opencv_func(*[img_uint8]+argv)
        if type(ret_img) == tuple:
            ret_img = ret_img[1]
        ret_img = ret_img.astype(np.float32) / 255.
        ret_imgs.append(ret_img)
    ret_imgs = np.stack(ret_imgs)

    if imgs.shape[3] == 1:
        ret_imgs = np.expand_dims(ret_imgs, axis=3)

    return 

In [116]:
def bit_depth_py(x, bits):
    precisions = 2**bits
    return reduce_precision_py(x, precisions)

In [117]:
def non_local_means_color_py(imgs, search_window, block_size, photo_render):
    import cv2
    ret_imgs = opencv_wrapper(imgs, cv2.fastNlMeansDenoisingColored, [None,photo_render,photo_render,block_size,search_window])
    return ret_imgs

In [118]:
m = nn.Softmax(dim=1)

In [119]:
def get_distance(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [120]:
def train_fs(model, dataset, X1, train_fpr):
    distances = get_distance(model, dataset, X1)
    selected_distance_idx = int(np.ceil(len(X1) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    #print ("Threshold value: %f" % threshold)
    return threshold

In [121]:
def get_distance_test(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [122]:
def compute_distance_test(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

# Functions for proposed methods

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms
from quantus.functions.perturb_func import baseline_replacement_by_indices
from art.attacks.evasion import CarliniLInfMethod
import torch.optim as optim
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
import warnings
warnings.filterwarnings('ignore')

In [124]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Approach 1: Input squeeze

In [125]:
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

def infer_attribution_axes(a_batch: np.ndarray, x_batch: np.ndarray)-> Sequence[int]:
    """
    Reference: quantus library
    Infers the axes in x_batch that are covered by a_batch.
    Parameters
    ----------
    x_batch: np.ndarray
        A np.ndarray which contains the input data that are explained.
    a_batch: np.ndarray
        An array which contains pre-computed attributions i.e., explanations.
    Returns
    -------
    np.ndarray
        The axes inferred.
    """
    # TODO: Adapt for batched processing.

    if a_batch.shape[0] != x_batch.shape[0]:
        raise ValueError(
            f"a_batch and x_batch must have same number of batches ({a_batch.shape[0]} != {x_batch.shape[0]})"
        )

    if a_batch.ndim > x_batch.ndim:
        raise ValueError(
            "Attributions need to have <= dimensions than inputs, but {} > {}".format(
                a_batch.ndim, x_batch.ndim
            )
        )

    # TODO: We currently assume here that the batch axis is not carried into the perturbation functions.
    a_shape = [s for s in np.shape(a_batch)[1:] if s != 1]
    x_shape = [s for s in np.shape(x_batch)[1:]]

    if a_shape == x_shape:
        return np.arange(0, len(x_shape))

    # One attribution value per sample
    if len(a_shape) == 0:
        return np.array([])

    x_subshapes = [
        [x_shape[i] for i in range(start, start + len(a_shape))]
        for start in range(0, len(x_shape) - len(a_shape) + 1)
    ]
    if x_subshapes.count(a_shape) < 1:

        # Check that attribution dimensions are (consecutive) subdimensions of inputs
        raise ValueError(
            "Attribution dimensions are not (consecutive) subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}".format(
                x_batch.shape, a_batch.shape
            )
        )
    elif x_subshapes.count(a_shape) > 1:

        # Check that attribution dimensions are (unique) subdimensions of inputs.
        # Consider potentially expanded dims in attributions.

        if a_batch.ndim == x_batch.ndim and len(a_shape) < a_batch.ndim:
            a_subshapes = [
                [np.shape(a_batch)[1:][i] for i in range(start, start + len(a_shape))]
                for start in range(0, len(np.shape(a_batch)[1:]) - len(a_shape) + 1)
            ]
            if a_subshapes.count(a_shape) == 1:

                # Inferring channel shape.
                for dim in range(len(np.shape(a_batch)[1:]) + 1):
                    if a_shape == np.shape(a_batch)[1:][dim:]:
                        return np.arange(dim, len(np.shape(a_batch)[1:]))
                    if a_shape == np.shape(a_batch)[1:][:dim]:
                        return np.arange(0, dim)

            raise ValueError(
                "Attribution axes could not be inferred for inputs of "
                "shape {} and attributions of shape {}".format(
                    x_batch.shape, a_batch.shape
                )
            )

        raise ValueError(
            "Attribution dimensions are not unique subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}."
            "Please expand attribution dimensions for a unique solution".format(
                x_batch.shape, a_batch.shape
            )
        )
    else:
        # Infer attribution axes.
        for dim in range(len(x_shape) + 1):
            if a_shape == x_shape[dim:]:
                return np.arange(dim, len(x_shape))
            if a_shape == x_shape[:dim]:
                return np.arange(0, dim)

    raise ValueError(
        "Attribution axes could not be inferred for inputs of "
        "shape {} and attributions of shape {}".format(x_batch.shape, a_batch.shape)
    )


In [126]:
def modify_x_based_on_attribution(x_batch, a_batch_benign, d_type, top_k = 100):
    #get attribution axes
    a_axes = infer_attribution_axes(a_batch_benign, x_batch)
    perturbed_arr = []

    #modify each sample one-by-one
    for x,a in zip(x_batch, a_batch_benign):
        #flatten the attribution 
        a = a.flatten()
        #get indices of sorted attributions (ascending)
        a_ind = np.argsort(a)
        #get indices of top_k 
        a_ix = a_ind[-top_k:]
        if d_type=="adv":
            x = x.cpu()
            x_perturbed = baseline_replacement_by_indices(arr = x.detach().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.detach().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
        else:
            x_perturbed = baseline_replacement_by_indices(arr = x.cpu().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.cpu().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
    new_batch = torch.stack(perturbed_arr)
    return new_batch.to('cuda')

## Approach 2: Adding noise and computing robustness

In [127]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [128]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)

# Approach 3: Statistics

In [129]:
def compute_mean_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        avg = np.mean(a)
        deviation = a - avg 
        absolute_deviation = np.abs(deviation)
        result = np.mean(absolute_deviation)
        scores.append(result)
    return scores    

In [130]:
def compute_median_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        med = np.median(a)
        deviation = a - med 
        abs_deviation = np.abs(deviation)
        result = np.median(abs_deviation)
        scores.append(result)
    return scores 

In [131]:
def compute_iqr(attr):
    #inter-quartile range
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = score_75 - score_25
        scores.append(score_qt)
    return scores
    

In [132]:
#relative measure of dispersion
def compute_coef_var(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        m = np.mean(a)
        st = np.std(attr[i])
        sc = m/st
        scores.append(sc)
    return scores

In [133]:
#relative measure of dispersion

## Coefficient of quartile dev

def compute_coef_iqr(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = (score_75 - score_25)/(score_75 + score_25)
        scores.append(score_qt)
    return scores
    

# Attack functions

In [134]:
def make_attack(x_batch, y_batch, eps, normal_model): 
    
    alpha = eps/10
    steps = int(alpha*eps)
    images_pgd = projected_gradient_descent(normal_model, x_batch, eps, alpha, steps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [135]:
def make_fgsm_attack(x_batch, y_batch, eps, normal_model): 
    
    images_pgd = fast_gradient_method(normal_model, x_batch, eps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [136]:
def make_cw_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

In [137]:
def make_bim_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

# Models

In [138]:
#for natural and adversarial LeNet Model 
class LeNet_normal(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [139]:
#from rev2.cifar10.model_utils import resnet50, CIFAR10_RESNET50_CKPT_PATH

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, out_keys=None):
        out = {}
        x = self.conv1(x)
        out["c1"] = x
        x = self.bn1(x)
        out["bn1"] = x
        x = F.relu(x)
        out["r1"] = x

        x = self.layer1(x)
        out["l1"] = x
        x = self.layer2(x)
        out["l2"] = x
        x = self.layer3(x)
        out["l3"] = x
        x = self.layer4(x)
        out["l4"] = x

        x = F.avg_pool2d(x, 4)
        out["gvp"] = x
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        out["fc"] = x

        if out_keys is None:
            return x
        res = {}
        for key in out_keys:
            res[key] = out[key]
        return res


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


def resnet50():
    return ResNet(Bottleneck, [3,4,6,3])


def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])


def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

## For each of the dataset, save the metrics computed for the proposed approaches or using feature squeeze method to a csv. Then after all the metrics are computed, use thresholds we have to compute TPR. 

# CIFAR

In [155]:
def compute_pgd_cifar(test_loader, normal_model, eps): 
    
    print("Computing metrics for {} for PGD".format(eps))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs =[]
    

    for step, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
            
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "cifar", images_adv)
        fs.extend(d)
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 400, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 450, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 500, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:
            
            df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 400",
            "Input squeeze 450",
            "Input squeeze 500",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "Feature squeezer",
        ])
            
            path = "eps_"+str(eps)+"_CIFAR_PGD.csv"
            df.to_csv(path)
            
            break

In [156]:
def compute_fgsm_cifar(test_loader, normal_model, eps): 
    
    print("Computing metrics for {} for PGD".format(eps))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs=[]

    for step, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_fgsm_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
            
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "cifar", images_adv)
        fs.extend(d)
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 400, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 450, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 500, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:
            
            df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 400",
            "Input squeeze 450",
            "Input squeeze 500",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "Feature squeezer",
        ])
            
            path = "eps_"+str(eps)+"_CIFAR_FGSM.csv"
            df.to_csv(path)
            
            break

In [157]:

def compute_bim_cifar(train_loader_cifar, normal_model, eps): 
    
    print("Computing metrics for {} for BIM".format(eps))
    
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(3, 32, 32),
        nb_classes=10,
)
    attack = BasicIterativeMethod(classifier, eps = eps, max_iter=int(eps*256*1.25))
    
    for step, (x_batch, y_batch) in enumerate(train_loader_cifar):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_bim_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "cifar", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 400, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 450, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 500, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:
            
            df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 400",
            "Input squeeze 450",
            "Input squeeze 500",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "Feature squeezer"
        ])
            
            path = "eps_"+str(eps)+"_CIFAR_BIM.csv"
            df.to_csv(path)
            
            break

In [158]:

def compute_cw_cifar(test_loader, normal_model, conf): 
    
    print("Computing metrics for {} for CW".format(conf))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(3, 32, 32),
        nb_classes=10,
)
    attack = CarliniLInfMethod(classifier, confidence = conf)
    
    for step, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_cw_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "cifar", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 400, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 450, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 500, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:
            
            df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 400",
            "Input squeeze 450",
            "Input squeeze 500",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "Feature squeezer"
        ])
            
            path = "confidence_"+str(conf)+"_CIFAR_CW.csv"
            df.to_csv(path)
            
            break

In [159]:
def load_cifar_model(path):
    model = resnet50()
    ckpt_dict = torch.load(path, lambda storage, loc: storage)
    model.load_state_dict(ckpt_dict)
    model.to('cuda')
    model.train(False)
    return model

In [160]:
def main_cifar(): 
    
    #load cifar model and set to eval
    path = "cifar.ckpt"
    normal_model = load_cifar_model(path)
    normal_model.to(device)
    normal_model.eval()
    
    #get dataset
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=torchvision.transforms.ToTensor())
    test_loader = DataLoader(testset, shuffle=True, batch_size=10)
    
    #compute all attack for each hyperparameter one by one 
    epsilons = [64/255]
    #confidences = [0,50]
    #compute_benign_cifar(train_loader, normal_model) 
    for eps in epsilons:
        compute_fgsm_cifar(test_loader, normal_model, eps)
        compute_pgd_cifar(test_loader, normal_model, eps)
        compute_bim_cifar(test_loader, normal_model, eps)
    
    #for conf in confidences: 
        #compute_cw_cifar(test_loader, normal_model, conf)


In [161]:
main_cifar()

Files already downloaded and verified
Computing metrics for 0.25098039215686274 for PGD
Computing metrics for 0.25098039215686274 for PGD
Computing metrics for 0.25098039215686274 for BIM


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# MNIST

In [162]:
def compute_pgd_mnist(test_loader, normal_model, eps): 
    
    print("Computing metrics for {} for PGD".format(eps))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []

    for step, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
        
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "mnist", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 100, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 150, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 180, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:            
            break
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 180",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
            "Feature squeeze"
        ])
            
    path = "eps_"+str(eps)+"_MNIST_PGD.csv"
    df.to_csv(path)

In [163]:
def compute_fgsm_mnist(test_loader, normal_model, eps): 
    
    print("Computing metrics for {} for PGD".format(eps))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []

    for step, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_fgsm_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
        
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "mnist", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 100, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 150, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 180, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:            
            break
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 180",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
            "Feature squeeze"
        ])
            
    path = "eps_"+str(eps)+"_MNIST_FGSM.csv"
    df.to_csv(path)

In [164]:
def compute_bim_mnist(train_loader_mnist, normal_model, eps): 
    
    print("Computing metrics for {} for BIM".format(eps))
    
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(1, 28, 28),
        nb_classes=10,
)
    attack = BasicIterativeMethod(classifier, eps = eps, max_iter=int(eps*256*1.25))
    
    for step, (x_batch, y_batch) in enumerate(train_loader_mnist):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_bim_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "mnist", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_max(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 100, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 150, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 180, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:
            break
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 180",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
            "Feature squeeze"
        ])
            
    path = "eps_"+str(eps)+"_MNIST_BIM.csv"
    df.to_csv(path)

In [165]:
def compute_cw_mnist(train_loader_mnist, normal_model, conf): 
    #compute metrics for svhn
    
    print("Computing metrics for {} for CW".format(conf))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(1, 28, 28),
        nb_classes=10,
)
    attack = CarliniLInfMethod(classifier, confidence = conf)
    
    for step, (x_batch, y_batch) in enumerate(train_loader_mnist):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_cw_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "mnist", images_adv)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 100, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 150, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 180, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 500:    
            break
    
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Median Absolute Dev", 
            "Mean Absolute Dev", 
            "IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 180",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
            "Feature squeeze"
        ])
            
    path = "confidence_"+str(conf)+"_MNIST_CW.csv"
    df.to_csv(path)

In [166]:
def load_mnist_model(path):
    model = LeNet_normal()
    model.to(device)
    model.load_state_dict(torch.load(path))
    model.to('cuda')
    model.train(False)
    return model

In [1]:
def main_mnist():
    
    
    #get model
    
    #get dataset
    test_set = torchvision.datasets.MNIST(root='./sample_data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, pin_memory=True)
    
    #compute all attack for each hyperparameter one by one 
    epsilons = [8/255, 16/255, 32/255, 64/255]
    confidences = [0,50]
    compute_benign_mnist(test_loader, normal_model) 
    for eps in epsilons:
        compute_fgsm_mnist(test_loader, normal_model, eps)
        compute_pgd_mnist(test_loader, normal_model, eps)
        compute_bim_mnist(test_loader, normal_model, eps)
    
    for conf in confidences: 
        compute_cw_mnist(test_loader, normal_model, conf)
        